In [80]:
import numpy as np
import pandas as pd
import os

In [81]:
df = pd.read_csv('../hier.csv')

def getLen(dataframe, level):
    length = df[f'level_{level}'].max() + 1
    return(length)

df['level_1'] = df['level_1'] + getLen(df, 0)
df['level_2'] = df['level_2'] + getLen(df, 1)
df['level_3'] = df['level_3'] + getLen(df, 2)
df['level_4'] = df['level_4'] + getLen(df, 3)
df['level_5'] = df['level_5'] + getLen(df, 4)

In [82]:
df

,gene,level_5,level_4,level_3,level_2,level_1,level_0
0,A1CF,397,129,26,14,1,0
1,AAAS,325,203,67,9,2,0
2,AAR2,542,126,43,14,1,0
3,AATF,598,149,43,14,1,0
4,ABAT,783,129,26,14,1,0
...,...,...,...,...,...,...,...
4963,ZW10,341,211,43,14,1,0
4964,ZWILCH,377,220,69,14,1,0
4965,ZYG11A,521,139,26,14,1,0
4966,ZYG11B,521,139,26,14,1,0


In [83]:
levels = [[],[],[],[],[],[]]
for i in range(0, 6):
    levels[i].extend(list(df[f'level_{i}'].unique()))
    levels[i].sort()

In [84]:
print('Size of each level')
for i, level in enumerate(levels):
    print(f'level {i}: {len(level)}')

Size of each level
level 0: 1
level 1: 2
level 2: 14
level 3: 53
level 4: 152
level 5: 657


In [85]:
levels

[[0],
 [1, 2],
 [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 [17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69],
 [70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,


In [99]:
genes = {}

for i, level in enumerate(levels):
    genes[f'level_{i}'] = {}
    for node in level:
        genes[f'level_{i}'][node] = list(df['gene'][df[f'level_{i}'] == node])

In [100]:
genes

{'level_0': {0: ['A1CF',
   'AAAS',
   'AAR2',
   'AATF',
   'ABAT',
   'ABCA2',
   'ABCB6',
   'ABCB8',
   'ABCB9',
   'ABCC8',
   'ABCC9',
   'ABCD4',
   'ABCG5',
   'ABCG8',
   'ABHD12',
   'ABHD6',
   'ABI1',
   'ABI2',
   'ABI3',
   'ABL1',
   'ABT1',
   'ABTB1',
   'ACD',
   'ACIN1',
   'ACR',
   'ACTA1',
   'ACTA2',
   'ACTB',
   'ACTC1',
   'ACTG1',
   'ACTG2',
   'ACTL10',
   'ACTL6A',
   'ACTL6B',
   'ACTL7A',
   'ACTL7B',
   'ACTL8',
   'ACTN4',
   'ACTR10',
   'ACTR1A',
   'ACTR1B',
   'ACTR2',
   'ACTR3',
   'ACTR5',
   'ACTR6',
   'ACTR8',
   'ACVR1',
   'ACVR1B',
   'ACVR1C',
   'ACVR2A',
   'ACVR2B',
   'ACVRL1',
   'ADAM10',
   'ADAM8',
   'ADAR',
   'ADCYAP1R1',
   'ADD1',
   'ADD2',
   'ADGRA2',
   'ADIPOQ',
   'ADNP',
   'ADRA2A',
   'ADRB2',
   'ADRB3',
   'ADRM1',
   'ADSL',
   'AEBP2',
   'AFDN',
   'AFF1',
   'AFF3',
   'AFF4',
   'AFG3L2',
   'AFTPH',
   'AGFG1',
   'AGK',
   'AGL',
   'AGO1',
   'AGO2',
   'AGO3',
   'AGO4',
   'AHCTF1',
   'AHI1',
   'AHNAK',

In [101]:
frames = []
for i in range(5):
    cols = df[[f'level_{i}', f'level_{i+1}']]
    cols = cols.drop_duplicates(subset=[f'level_{i}', f'level_{i+1}'], keep='first')
    cols.rename(columns={f'level_{i}': 'parent', f'level_{i+1}': 'child'}, inplace=True)

    frames.append(cols)

exp = pd.concat(frames)
print(exp.sort_values('parent'))

      parent  child
0          0      1
1          0      2
0          1     14
3417       2     15
3385       2     12
...      ...    ...
467      220    285
3632     220    786
4582     220    875
376      220    269
4935     221    877

[878 rows x 2 columns]


In [102]:
gene_dict = {
    **genes['level_0'],
    **genes['level_1'],
    **genes['level_2'],
    **genes['level_3'],
    **genes['level_4'],
    **genes['level_5'],
}
gene_ser = pd.Series(gene_dict)
gene_df = pd.DataFrame({'child':gene_ser.index, 'genes':gene_ser.values})
gene_ser[14]

['A1CF',
 'AAR2',
 'AATF',
 'ABAT',
 'ABCA2',
 'ABCB6',
 'ABCB8',
 'ABCB9',
 'ABCC8',
 'ABCC9',
 'ABCD4',
 'ABCG5',
 'ABCG8',
 'ABHD12',
 'ABHD6',
 'ABI1',
 'ABI2',
 'ABI3',
 'ABL1',
 'ABT1',
 'ABTB1',
 'ACD',
 'ACIN1',
 'ACR',
 'ACTL10',
 'ACTL6A',
 'ACTL6B',
 'ACTL7A',
 'ACTL7B',
 'ACTL8',
 'ACTN4',
 'ACTR10',
 'ACTR1A',
 'ACTR1B',
 'ACTR2',
 'ACTR3',
 'ACTR5',
 'ACTR6',
 'ACTR8',
 'ACVR1',
 'ACVR1B',
 'ACVR1C',
 'ACVR2A',
 'ACVR2B',
 'ACVRL1',
 'ADAM10',
 'ADAM8',
 'ADAR',
 'ADCYAP1R1',
 'ADD1',
 'ADD2',
 'ADGRA2',
 'ADIPOQ',
 'ADNP',
 'ADRA2A',
 'ADRB2',
 'ADRB3',
 'ADSL',
 'AEBP2',
 'AFDN',
 'AFF1',
 'AFF3',
 'AFF4',
 'AFG3L2',
 'AFTPH',
 'AGFG1',
 'AGK',
 'AGO1',
 'AGO2',
 'AGO3',
 'AGO4',
 'AHCTF1',
 'AHI1',
 'AHR',
 'AHRR',
 'AHSP',
 'AICDA',
 'AIM2',
 'AIMP2',
 'AIP',
 'AJUBA',
 'AKAP10',
 'AKAP17A',
 'AKAP4',
 'AKAP5',
 'AKAP6',
 'AKAP7',
 'AKAP9',
 'AKIRIN2',
 'AKT1',
 'AKT2',
 'AKTIP',
 'ALB',
 'ALG14',
 'ALK',
 'ALS2',
 'ALX4',
 'AMFR',
 'AMHR2',
 'AMN',
 'AMN1',
 'ANAPC1'

In [103]:
exp['genes'] = ''
exp['num_genes'] = 0
print(exp)

      parent  child genes  num_genes
0          0      1                0
1          0      2                0
0          1     14                0
1          2      9                0
25         2     13                0
...      ...    ...   ...        ...
4893     205    870                0
4911     218    872                0
4924     219    873                0
4935     221    877                0
4958      85    878                0

[878 rows x 4 columns]


In [104]:
children = gene_dict.keys()
for child in children:
    exp.loc[exp['child'] == child, 'num_genes'] = len(gene_dict[child])
    if child in gene_dict.keys():
        exp.loc[exp['child'] == child, 'genes'] = ':'.join(gene_dict[child])

exp

,parent,child,genes,num_genes
0,0,1,A1CF:AAR2:AATF:ABAT:ABCA2:ABCB6:ABCB8:ABCB9:AB...,4252
1,0,2,AAAS:ACTA1:ACTA2:ACTB:ACTC1:ACTG1:ACTG2:ADRM1:...,716
0,1,14,A1CF:AAR2:AATF:ABAT:ABCA2:ABCB6:ABCB8:ABCB9:AB...,4252
1,2,9,AAAS:AMBRA1:ARPC1A:ARPC1B:ATG16L1:ATG16L2:BOP1...,86
25,2,13,ACTA1:ACTA2:ACTB:ACTC1:ACTG1:ACTG2:ANAPC11:AP1...,217
...,...,...,...,...
4893,205,870,XPC,1
4911,218,872,YPEL5,1
4924,219,873,ZC3H8,1
4935,221,877,ZFP36:ZNF10:ZNF224:ZNF350,4


In [92]:
"""
# Export list of genes to csv file
exp.to_csv("cytoscape_input.csv", index=False, header=True)
"""